In [1]:
%run 01_Model_CFE.ipynb


c:\Users\Skole\AppData\Local\Programs\Python\Python39\lib\site-packages\nbformat\__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


ADS: frdp_c --> AMOR + ppi_c
CPR1_CYP71AV1: AMOR + h_c + nadph_c + o2_c --> AAOH + h2o_c + nadp_c
ADH1: AAOH + nad_c --> AAld + h_c + nadh_c
ALDH1: AAld + h2o_c + nadp_c --> AA + h_c + nadph_c


In [2]:
from cameo import models


In [3]:
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.objective_functions import biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva
import cameo

In [4]:
modelS=iMM904.copy()
#changing source to sucrose
medium09 = modelS.medium
medium09['EX_sucr_e'] = 10
medium09['EX_glc__D_e'] = 0
modelS.reactions.EX_sucr_e.lower_bound=-10
modelS.medium = medium09
modelS.objective = modelS.reactions.ALDH1


In [5]:
modelS.reactions.ADS.lower_bound = 0
modelS.reactions.CPR1_CYP71AV1.lower_bound = 0
modelS.reactions.ADH1.lower_bound = 0
modelS.reactions.ALDH1.lower_bound = 0

modelS.reactions.EX_glc__D_e.lower_bound = -10
modelS.reactions.EX_o2_e.lower_bound = -10

In [6]:
modelS.reactions.BIOMASS_SC5_notrace.lower_bound = 0.1


In [7]:
modelS.objective = modelS.reactions.ALDH1

In [8]:
py = product_yield(modelS.reactions.ALDH1, modelS.reactions.EX_sucr_e)
optswap = CofactorSwapOptimization(model=modelS, objective_function=py, cofactor_id_swaps=(['nadp_c', 'nadph_c'],['nad_c', 'nadh_c'] ))

In [14]:
optswap.run(max_evaluations=2000, max_size=5)

Starting optimization at Fri, 01 Dec 2023 10:57:27


HBox()

Finished after 00:02:20


,index,targets,fitness
0,0,"(GAPD,)",1.034066
1,2,"(ALCD2x_copy1,)",1.034066
2,4,"(ALCD2x_copy2,)",1.034066
3,10,"(HMGCOAR, ADH1)",1.031084
4,11,"(ADH1, HMGCOAR)",1.024722
...,...,...,...
61,94,"(FAS80COA_L, C4STMO1)",0.840317
62,95,"(FAS180, IPMD)",0.840314
63,96,"(FAS80COA_L,)",0.840311
64,98,"(SHK3Dr, TRDR)",0.840309


In [15]:
print('Original model yield' ,modelS.optimize().objective_value, 'mmol_AA/gDW*h')

print('Cofactor swapped model yield',optswap.model.optimize().objective_value, 'mmol_AA/gDW*h')


Original model yield 2.8000189646464633 mmol_AA/gDW*h
Cofactor swapped model yield 2.8000189646464637 mmol_AA/gDW*h


In [16]:
optswap.model.reactions.ALDH1.model = optswap.model
optswap.model.objective = optswap.model.reactions.ALDH1

AttributeError: can't set attribute

In [ ]:
original = (optswap.model.solve().f * 4) / (-optswap.model.reactions.EX_sucr_e.flux * 6)


AttributeError: 'Model' object has no attribute 'solve'

In [ ]:
with TimeMachine() as tm:
    optswap.model.swap_reaction('GAPD', tm)
    swapped = (optswap.model.solve().f * 4) / (-optswap.model.reactions.EX_sucr_e.flux * 6)

print("product/substrate yield without swap: {}\nproduct/substrate yield with swap: {}".format(original, swapped))

In [ ]:
print("product/substrate yield without swap: {}\nproduct/substrate yield with swap: {}".format(original, swapped))

NameError: name 'original' is not defined